In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
import numpy as np
import random as rand
import qiskit
import timeit
from qiskit.providers.ibmq.managed import IBMQJobManager

In [3]:
## common code
def error(x, x_estimate):
    return np.linalg.norm(x - x_estimate) / np.linalg.norm(x)

def binarylist_to_int(binary_list):
    
    sum = 0
    
    two_power = 1
    
    for digit in binary_list[-1::-1]:
        sum += two_power * digit
        
        two_power *= 2
    
    return sum



def int_to_binarylist(integer, num_digits):
    """
    integer: int
    
    num_digits: int, at least 1
    
    returns a list of length num_digits, including leading 0s if necessary
    """
    
    if num_digits < len(bin(integer)) - 2:
        raise Exception("num_digits too short")

    
    binary_list = []
    
    for i in range(num_digits)[::-1]:
        two_power = 2**(i)
        
        binary_list.append(int(integer/two_power))
        
        integer -= int(integer/two_power) * two_power
        
    return binary_list

# Classical Random Walk LSP

In [10]:
class ClassicalRandomWalkHamming:
    
    """
    code for classical random walk (?) 
    """
    
    import numpy as np
    import random as rand
    
    def __init__(self, theta_list):
        
        """
        theta_list: list of angles, whose cos and sin correspond to non-flip and flip probabilities
            [theta_n-1, theta_n-2, ..., theta_1, theta_0]
        
        """
        
        self.theta_list = theta_list
        
        self.n = len(theta_list)
        
    def p_matrix(self):
        """
        generates and returns P matrix denoted by thetas
        
        the J', J-th element is defined by prod_{l=0,...,n-1} cos^2(theta_l/2)^(1-i_l) * sin^2(theta_l/2)^i_l
        
        """
        
        matrix = np.ones((2**self.n, 2**self.n)) # start with one, to be multiplied
        
        for i in range(2**self.n):
            for j in range(2**self.n):
                
                xor_int = i ^ j
                
                xor_binary = int_to_binarylist(xor_int, self.n)
                
                for idx, digit in enumerate(xor_binary):
                    if digit == 1: # flip
                        matrix[i,j] *= np.sin(self.theta_list[idx]/2)**2
                    elif digit == 0:
                        matrix[i,j] *= np.cos(self.theta_list[idx]/2)**2
                    else:
                        raise Exception("Value error")
        
        return matrix 
    
        
    def random_step(self, current_node):
        """
        current_node: list of 0s and 1s [b_n-1, b_n-2, ..., b_1, b_0]
        
        iterates through each digit and flips according to probabilities given by thetas
        
        returns a list of 0s and 1s
        """
        
        def bit_flip(one_or_zero):
            if one_or_zero == 0:
                return 1
            elif one_or_zero == 1:
                return 0
            else:
                raise Exception("Value not 0 or 1")
                
#         for idx, theta in enumerate(self.theta_list):
#             rand_val = rand.random() # simulate coin flip with value between 0 and 1
            
#             if rand_val < (np.sin(theta/2))**2: # flip
#                 current_node[idx] = bit_flip(current_node[idx])
                
#             else: # if rand_val < (np.sin(theta))**2 + (np.cos(theta))**2
#                 pass # don't flip

        for idx, theta in enumerate(self.theta_list): # start with theta_{n-1}, end with theta_0
            rand_val = rand.random() # generate random value between 0 and 1
            
            if rand_val > (np.sin(theta/2))**2:    
                pass
            else: # if rand_val < (np.sin(theta))**2) + np.cos((theta))**2
                current_node[idx] = bit_flip(current_node[idx])
        
        return current_node
        
    def random_walk(self, initial_node, num_steps):
        """
        initial_node: list of 0s and 1s [b_n-1, b_n-2, ..., b_1, b_0]
        
        returns a list of 0s and 1s
        """
        
        current_node = initial_node
        
        for step in range(num_steps):
            current_node = self.random_step(current_node)
            
        return current_node
    
    
def x_component_crw(theta_list, gamma, b, x_component_index, c, d):
    """
    theta_list: list of angles, whose cos and sin correspond to non-flip and flip probabilities
            [theta_n-1, theta_n-2, ..., theta_1, theta_0]
        
        for A = 1 - gamma*P,
        
        A @ x = b
        
        A_inv = sum_{s=0,...,inf} (gamma^s * P^s)
        A_inv_c = sum_{s=0,...,c} (gamma^s * P^s)
        
        x_c = A_inv_c @ b
        
    b: 1D list or np.array representing end result vector
    x_component_index: int
    c: int; truncation of series (deptH)
    d: int; number of times to perform each step (num_shots)
    
    returns the appropriate index value of x_c (truncated up to error ~ gamma^c)
    """
    crw = ClassicalRandomWalkHamming(theta_list)
    
    prep_node = int_to_binarylist(x_component_index, num_digits = crw.n)
    
    x_component = 0
    
    for s in range(0, c+1): # number of steps starting at 0
        
        total_sum = 0
        
        for iter in range(d):
            final_node = crw.random_walk(prep_node, s)
            final_node_int = binarylist_to_int(final_node)
            total_sum += b[final_node_int]
                    
        avg_sum = total_sum / d
        
        avg_sum *= gamma**s
        
        ## add to component value
        x_component += avg_sum
        
    return x_component

### 32 x 32 Random Matrix Test

In [11]:
n = 5
theta_list = [rand.random()*np.pi for i in range(n)] ## generate random angles

crw = ClassicalRandomWalkHamming(theta_list)
P = crw.p_matrix() ## get stochastic matrix

gamma = 0.5 * rand.random() # multiplicative constant between 0 and 1 - made smaller for illustrative purposes

A = np.eye(2**n) - gamma*P
b = np.array([-1+2*rand.random() for i in range(2**n)]) ## generate random vector
x = np.linalg.inv(A) @ b ## calculate exact solution

In [12]:
print(gamma)
print(P)
print(A)
print(b)
print(x)

0.32903943794434065
[[0.00066927 0.07664998 0.00018832 ... 0.07114605 0.00017479 0.02001869]
 [0.07664998 0.00066927 0.02156735 ... 0.00062121 0.02001869 0.00017479]
 [0.00018832 0.02156735 0.00066927 ... 0.02001869 0.00062121 0.07114605]
 ...
 [0.07114605 0.00062121 0.02001869 ... 0.00066927 0.02156735 0.00018832]
 [0.00017479 0.02001869 0.00062121 ... 0.02156735 0.00066927 0.07664998]
 [0.02001869 0.00017479 0.07114605 ... 0.00018832 0.07664998 0.00066927]]
[[ 9.99779784e-01 -2.52208672e-02 -6.19633200e-05 ... -2.34098561e-02
  -5.75139780e-05 -6.58693790e-03]
 [-2.52208672e-02  9.99779784e-01 -7.09651036e-03 ... -2.04403619e-04
  -6.58693790e-03 -5.75139780e-05]
 [-6.19633200e-05 -7.09651036e-03  9.99779784e-01 ... -6.58693790e-03
  -2.04403619e-04 -2.34098561e-02]
 ...
 [-2.34098561e-02 -2.04403619e-04 -6.58693790e-03 ...  9.99779784e-01
  -7.09651036e-03 -6.19633200e-05]
 [-5.75139780e-05 -6.58693790e-03 -2.04403619e-04 ... -7.09651036e-03
   9.99779784e-01 -2.52208672e-02]
 [-6.5

In [13]:
## get solution with random walk
start = timeit.default_timer()
### timing
depth = 6 
num_shots = 5000 ## num_shots

x_estimate = np.array([x_component_crw(theta_list, gamma, b, x_component_index, depth, num_shots) 
                      for x_component_index in range(2**n)]) ## get solution vector with CRW

###########
end = timeit.default_timer()
print(f"time elapsed: {end-start}")

time elapsed: 29.29904199996963


In [14]:
## getting error rate
error(x, x_estimate)

0.0685673645532875

# Quantum Random Walk LSP - With Simulated Backend

In [31]:
def int_to_binarylist(integer, num_digits):
    """
    integer: int
    
    num_digits: int, at least 1
    
    returns a list of length num_digits, including leading 0s if necessary
    """
    
    if num_digits < len(bin(integer)) - 2:
        raise Exception("num_digits too short")

    
    binary_list = []
    
    for i in range(num_digits)[::-1]:
        two_power = 2**(i)
        
        binary_list.append(int(integer/two_power))
        
        integer -= int(integer/two_power) * two_power
        
    return binary_list


class QuantumRandomWalkHamming:
    
    import numpy as np
    import qiskit
    
    def __init__(self, angle_list):
        """
        angle_list: list of [theta, phi, lambda] triples, starting with index n-1 and ending with 0 (left to right)
        
        """
        self.angle_list = angle_list
        
        self.n = len(angle_list)
       
    
    def p_matrix(self):
        """
        generates the appropriate numpy matrix
        
        """
        matrix = np.ones( (2**self.n, 2**self.n) ) # start with ones, values will be multiplied
        
        for i in range(2**self.n): # rows
            for j in range(2**self.n): # columns
                
                xor_int = i  ^ j
                
                xor_binarylist = int_to_binarylist(xor_int, self.n)
                
                # test pairs n-1,n-2; n-2,n-3; ...; 1,0
                for idx in range(len(xor_binarylist) - 1):
                    
                    power = xor_binarylist[idx] ^ xor_binarylist[idx+1]
                    
                    if power == 1:
                        matrix[i][j] *= np.sin(self.angle_list[idx][0]/2)**2 # sin(theta_idx/2) ^ 2
                    elif power == 0:
                        matrix[i][j] *= np.cos(self.angle_list[idx][0]/2)**2 # cos(theta_idx/2) ^ 2
                    else:
                        raise Exception("Value error")
                        
                # test pair 0,-1
                idx = len(xor_binarylist) - 1
                power = xor_binarylist[idx]
                
                if power == 1:
                    matrix[i][j] *= np.sin(self.angle_list[idx][0]/2)**2
                elif power == 0:
                    matrix[i][j] *= np.cos(self.angle_list[idx][0]/2)**2
                else:
                    raise Exception("Value error")
                
        return matrix
            
        
    def random_step(self, initial_node):
        """
        initial_node: an int as a binary_list
        
        returns the final node as a binary_list
        """
        
        ## instantiate circuit stuff
        qr = qiskit.QuantumRegister(self.n + 1) # nth index qubit is coin 
        cr = qiskit.ClassicalRegister(self.n)
        
        qc = qiskit.QuantumCircuit(qr, cr)
        
        ## initialize registers according to initial_node
        for idx, bit in enumerate(initial_node[-1::-1]):
            if bit == 1:
                qc.x(idx)
            elif bit == 0:
                pass
            else:
                raise Exception("Value error")
        
        
        ## perform coin flip unitary operations and cnots
        # iterate from back to front, since self.angle_list goes from n-1 to 0
        for idx, triple in enumerate(self.angle_list[-1::-1]):
            
            # apply u3 to coin qubit
            qc.u(triple[0], triple[1], triple[2], self.n) # (theta, phi, lambda, coin_idx)
            
            # apply cnot, with control on coin qubit, target on idx-th qubit 
            qc.cnot(self.n, idx) # (control, target)
        
        # apply measurement operation
        qc.measure(qr[0:self.n], cr[0:self.n]) # exclude coin qubit
        
        ## readout final result, then convert to binary_list
        # use qasm simulator by default
        result = qiskit.execute(qc, backend = qiskit.Aer.get_backend('qasm_simulator'), shots = 1).result()
        counts = result.get_counts()
        
#         print(counts)
        
        # convert dict_keys to list, then get key
        final_node_str = list(counts.keys())[0] # returned as str
        
        final_node_list = [digit_char for digit_char in final_node_str]
        
        final_node_binarylist = [int(digit_char) for digit_char in final_node_list]
        
        return final_node_binarylist
        
    def random_walk(self, initial_node, num_steps):
        """
        initial_node: an int as a binary_list
        num_steps: int
        
        """
        
        current_node = initial_node
        
        for step in range(num_steps):
            
            current_node = self.random_step(current_node)
            
        return current_node
    
    
    
## code for qrw linear solver
def x_component_qrw(angle_list, gamma, b, x_component_index, c, d):
    """
    angle_list: list of [theta, phi, lambda] triples that correspond to coin unitaries
                    listed n-1, n-2, ..., 0 from left to right
        
        for A = 1 - gamma*P,
        
        A @ x = b
        
        A_inv = sum_{s=0,...,inf} (gamma^s * P^s)
        A_inv_c = sum_{s=0,...,c} (gamma^s * P^s)
        
        x_c = A_inv_c @ b
        
    b: 1D list or np.array representing end result vector
    x_component_index: int
    c: int; cutoff 
    d: int; number of times to perform each step - a value from 100 to 10000 can be used 
    
    returns the appropriate index value of x_c (truncated up to error ~ gamma^c)
    """
    qrw = QuantumRandomWalkHamming(angle_list)
    
    prep_node = int_to_binarylist(x_component_index, num_digits = qrw.n)
    
    x_component = 0
    
    for s in range(0, c+1): # number of steps starting at 0
        
        total_sum = 0
        
        for iter in range(d):
            final_node = qrw.random_walk(prep_node, s)
            final_node_int = binarylist_to_int(final_node)
            total_sum += b[final_node_int]
                    
        avg_sum = total_sum / d
        
        avg_sum *= gamma**s
        
        ## add to component value
        x_component += avg_sum
        
    return x_component

### 4 x 4 Random Matrix Test

In [32]:
n = 2 # for 2^2 = 4

angle_list = [(rand.random()*np.pi, 0, 0) for i in range(n)] ## generate random triplets

qrw = QuantumRandomWalkHamming(angle_list)
P = qrw.p_matrix()

gamma = 0.5 * rand.random() # multiplicative constant between 0 and 1 - made smaller for illustrative purposes

A = np.eye(2**n) - gamma*P
b = np.array([-1+2*rand.random() for i in range(2**n)])
x = np.linalg.inv(A) @ b

In [33]:
print(gamma)
print(P)
print(A)
print(b)
print(x)

0.1666172355114866
[[0.82203621 0.00660186 0.0419287  0.12943324]
 [0.00660186 0.82203621 0.12943324 0.0419287 ]
 [0.0419287  0.12943324 0.82203621 0.00660186]
 [0.12943324 0.0419287  0.00660186 0.82203621]]
[[ 0.8630346  -0.00109998 -0.00698604 -0.02156581]
 [-0.00109998  0.8630346  -0.02156581 -0.00698604]
 [-0.00698604 -0.02156581  0.8630346  -0.00109998]
 [-0.02156581 -0.00698604 -0.00109998  0.8630346 ]]
[-0.44112706  0.17299481 -0.4602122  -0.20322823]
[-0.5214052   0.18445667 -0.53317591 -0.24769647]


In [34]:
## get solution with random walk
start = timeit.default_timer()
### timing

depth = 6 
num_shots = 100 ## num_shots

x_estimate = np.array([x_component_qrw(angle_list, gamma, b, x_component_index, depth, num_shots) 
                  for x_component_index in range(2**n)])

###########
end = timeit.default_timer()
print(f"time elapsed: {end-start}")

time elapsed: 32.44782046799082


In [35]:
## getting error rate
error(x, x_estimate)

0.009133708604856367

# Quantum Random Walk LSP - With Real Backend
Note: this might take a while to execute on real systems due to long queue wait times

In [36]:
class QuantumRandomWalkHamming:
    
    import numpy as np
    import qiskit
    
    def __init__(self, angle_list):
        """
        angle_list: list of [theta, phi, lambda] triples, starting with index n-1 and ending with 0 (left to right)
        
        """
        self.angle_list = angle_list
        
        self.n = len(angle_list)
       
    
    def p_matrix(self):
        """
        generates the appropriate numpy matrix
        
        """
        matrix = np.ones( (2**self.n, 2**self.n) ) # start with ones, values will be multiplied
        
        for i in range(2**self.n): # rows
            for j in range(2**self.n): # columns
                
                xor_int = i  ^ j
                
                xor_binarylist = int_to_binarylist(xor_int, self.n)
                
                # test pairs n-1,n-2; n-2,n-3; ...; 1,0
                for idx in range(len(xor_binarylist) - 1):
                    
                    power = xor_binarylist[idx] ^ xor_binarylist[idx+1]
                    
                    if power == 1:
                        matrix[i][j] *= np.sin(self.angle_list[idx][0]/2)**2 # sin(theta_idx/2) ^ 2
                    elif power == 0:
                        matrix[i][j] *= np.cos(self.angle_list[idx][0]/2)**2 # cos(theta_idx/2) ^ 2
                    else:
                        raise Exception("Value error")
                        
                # test pair 0,-1
                idx = len(xor_binarylist) - 1
                power = xor_binarylist[idx]
                
                if power == 1:
                    matrix[i][j] *= np.sin(self.angle_list[idx][0]/2)**2
                elif power == 0:
                    matrix[i][j] *= np.cos(self.angle_list[idx][0]/2)**2
                else:
                    raise Exception("Value error")
                
        return matrix
            
        
    def random_step(self, initial_node):
        """
        initial_node: an int as a binary_list
        
        returns the final node as a binary_list
        """
        
        ## instantiate circuit stuff
        qr = qiskit.QuantumRegister(self.n + 1) # nth index qubit is coin 
        cr = qiskit.ClassicalRegister(self.n)
        
        qc = qiskit.QuantumCircuit(qr, cr)
        
        ## initialize registers according to initial_node
        for idx, bit in enumerate(initial_node[-1::-1]):
            if bit == 1:
                qc.x(idx)
            elif bit == 0:
                pass
            else:
                raise Exception("Value error")
        
        
        ## perform coin flip unitary operations and cnots
        # iterate from back to front, since self.angle_list goes from n-1 to 0
        for idx, triple in enumerate(self.angle_list[-1::-1]):
            
            # apply u3 to coin qubit
            qc.u(triple[0], triple[1], triple[2], self.n) # (theta, phi, lambda, coin_idx)
            
            # apply cnot, with control on coin qubit, target on idx-th qubit 
            qc.cnot(self.n, idx) # (control, target)
        
#         # apply measurement operation
        qc.measure(qr[0:self.n], cr[0:self.n]) # exclude coin qubit
                ### note: previously I forgot to uncomment this 
        
        ### unused code from qasm_simulator implementation        
#         ## readout final result, then convert to binary_list
#         # use qasm simulator by default
#         result = qiskit.execute(qc, backend = qiskit.Aer.get_backend('qasm_simulator'), shots = 1).result()
#         counts = result.get_counts()
        
# #         print(counts)
        
#         # convert dict_keys to list, then get key
#         final_node_str = list(counts.keys())[0] # returned as str
        
#         final_node_list = [digit_char for digit_char in final_node_str]
        
#         final_node_binarylist = [int(digit_char) for digit_char in final_node_list]
        
#         return final_node_binarylist

        return qc
        
    def random_walk(self, initial_node, num_steps):
        """
        initial_node: an int as a binary_list
        num_steps: int
        
        """
        
        current_node = initial_node
        
        for step in range(num_steps):
            
            current_node = self.random_step(current_node)
            
        return current_node
    
## code for qrw linear solver
def x_component_qrw(angle_list, gamma, b, x_component_index, c, d, backend_obj):
    """
    angle_list: list of [theta, phi, lambda] triples that correspond to coin unitaries
                    listed n-1, n-2, ..., 0 from left to right
        
        for A = 1 - gamma*P,
        
        A @ x = b
        
        A_inv = sum_{s=0,...,inf} (gamma^s * P^s)
        A_inv_c = sum_{s=0,...,c} (gamma^s * P^s)
        
        x_c = A_inv_c @ b
        
    b: 1D list or np.array representing end result vector
    x_component_index: int
    c: int; cutoff 
    d: int; number of times to perform each step - a value from 100 to 10000 can be used 
    
    returns the appropriate index value of x_c (truncated up to error ~ gamma^c)
    """
    from qiskit.providers.ibmq.managed import IBMQJobManager
    import time
    
    qrw = QuantumRandomWalkHamming(angle_list)
    
    prep_node = int_to_binarylist(x_component_index, num_digits = qrw.n)
    
    x_component = 0 ## final value to be returned
    
    walk_nodes = [prep_node for i in range(d)]
    
    
    ## total number of batches
    completed_jobs = 0
    total_jobs = 0
    for s in range(1, c+1):
        for sub_step in range(s):
            total_jobs += int(np.ceil(d/75))
    
    ## for 0 steps
    total_sum = 0
    
    for i in range(d):
        final_node_list = walk_nodes[i]

        final_node_int = binarylist_to_int(final_node_list)
        total_sum += b[final_node_int]
                    
        avg_sum = total_sum / d
        
        avg_sum *= gamma**0
        
    x_component += avg_sum
    
    
    for s in range(1, c+1): # number of steps starting at 1
        
        total_sum = 0
        
        walk_nodes = [prep_node for i in range(d)]
        
        for sub_step in range(s): ## don't reuse steps 
        
            ## submit jobs
            qc_list = []

            for node in walk_nodes:
                qc_list.append(qrw.random_step(node))

            transpiled_thing = qiskit.transpile(qc_list, backend = backend_obj)

            job_manager = IBMQJobManager()

            job_set = job_manager.run(transpiled_thing, backend = backend_obj, shots = 1)

            ## get results
            start = timeit.default_timer()
            while True:
                time.sleep(2)

                completed_idx = job_set.report().find('Successful jobs: ')
    #             queue_idx = job_set.report().find('queue position: ')

                current_completed_jobs = job_set.report()[completed_idx:][len('Successful jobs: ')]

    #             if queue_idx == -1:
    #                 queue_position = -1
    #             else:
    #                 queue_position = job_set.report()[queue_idx:][len('queue position: ')]

                ## for each step, 1st number is # of batches completed, 2nd number is # of total batches, 3rd is time elapsed
#                 print('\r', current_completed_jobs, str(int(np.ceil(d/75))), (timeit.default_timer()-start), end = '', flush = True)
#                 print('\r', str(completed_batches+int(current_completed_jobs)), 
#                       str(total_batches), (timeit.default_timer()-start), end = '', flush = True)
                print(f"\r jobs done: {completed_jobs+int(current_completed_jobs)}/{total_jobs}, current batch time: {timeit.default_timer()-start}", end = '', flush = True)
                if current_completed_jobs == str(int(np.ceil(d/75))):
                    completed_jobs += int(current_completed_jobs)
                    break # jobs are finished

            results = job_set.results()

            for i in range(d):
                final_node_str = list(results.get_counts(i).keys())[0]
                final_node_list = [int(digit_char) for digit_char in final_node_str]

                # update nodes
                walk_nodes[i] = final_node_list.copy()
            
        ## calculate addend at the end
        for i in range(d):
            final_node_str = list(results.get_counts(i).keys())[0]
            final_node_list = [int(digit_char) for digit_char in final_node_str]
            
            final_node_int = binarylist_to_int(final_node_list)
            total_sum += b[final_node_int]
                    
        avg_sum = total_sum / d
        
        avg_sum *= gamma**s
        
        ## add to component value
        x_component += avg_sum
        
    return x_component

### 4 x 4 Random Matrix Test

In [37]:
n = 2 # for 2^2 = 4

angle_list = [(rand.random()*np.pi, 0, 0) for i in range(n)] ## generate random triplets

qrw = QuantumRandomWalkHamming(angle_list)
P = qrw.p_matrix()

gamma = 0.5 * rand.random() # multiplicative constant between 0 and 1 - made smaller for illustrative purposes

A = np.eye(2**n) - gamma*P
b = np.array([-1+2*rand.random() for i in range(2**n)])
x = np.linalg.inv(A) @ b

In [38]:
print(gamma)
print(P)
print(A)
print(b)
print(x)

0.10963546816737957
[[0.83965326 0.00657658 0.05715513 0.09661503]
 [0.00657658 0.83965326 0.09661503 0.05715513]
 [0.05715513 0.09661503 0.83965326 0.00657658]
 [0.09661503 0.05715513 0.00657658 0.83965326]]
[[ 9.07944222e-01 -7.21026177e-04 -6.26622996e-03 -1.05924338e-02]
 [-7.21026177e-04  9.07944222e-01 -1.05924338e-02 -6.26622996e-03]
 [-6.26622996e-03 -1.05924338e-02  9.07944222e-01 -7.21026177e-04]
 [-1.05924338e-02 -6.26622996e-03 -7.21026177e-04  9.07944222e-01]]
[ 0.81120779 -0.37702974  0.63761067  0.11690127]
[ 0.89958965 -0.40538519  0.70384553  0.13700992]


In [ ]:
## get solution with random walk
start = timeit.default_timer()
### timing

backend_string = 'ibmq_santiago' ## change according to backend
backend = provider.get_backend(backend_string)

depth = 6 
num_shots = 100 ## num_shots

x_estimate = np.array([0 for idx in range(2**n)])
for x_component_index in range(2**n):
    print(f"component #{x_component_idx}:")
    x_estimate[x_component_index] = x_component_qrw(angle_list, gamma, b, x_component_index, depth, num_shots, backend)
    print()

###########
end = timeit.default_timer()
print(f"time elapsed: {end-start}")

component #0:
 jobs done: 27/42, current batch time: 3860.4572056260662

In [ ]:
## getting error rate
error(x, x_estimate)